In [9]:
# This file is used to test filtering/searching for key terms in a csv
# It uses the 'program.csv' file from the DSIRE data
# The individual program IDs and Names are listed as comments at the
#     end of this document
# I am still working on how to directly filter out the empty dataframes
#     from the data in order to clean it
# The results listed include the programs where at least one
#     search term is listed at least once, but does not differentiate
#     which search term is listed. Each term can also be searched
#     individually with code that is commented out if that is preferred.

In [2]:
import numpy as np
import pandas as pd
import json
from functools import reduce
from operator import itemgetter

In [3]:
programdata = pd.read_csv(r'/Users/Jax/Desktop/Research with Evan/dsire-2020-07/program.csv', index_col=0)
programdata

,state_id,is_entire_state,implementing_sector_id,program_category_id,program_type_id,created_by_user_id,code,name,updated_ts,created_ts,...,administrator,fundingsource,budget,start_date,start_date_text,end_date,end_date_text,summary,additional_technologies,fromSir
id,,,,,,,,,,,,,,,,,,,,,
1,39,1,1,2,44,534,ND02R,Solar Easements,2020-06-01 17:35:33,2000-01-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<p>&#10;&#9;North Dakota's solar easement law ...,NaN,0
2,4,1,1,1,32,534,AZ10F,Qualifying Wood Stove Deduction,2017-03-21 20:07:14,2000-01-01 00:00:00,...,NaN,NaN,NaN,1994-01-01 05:00:00,1/1/1994,NaN,NaN,<div>&#10;&#9;This incentive allows Arizona ta...,NaN,0
3,24,1,1,2,44,534,MD01R,Solar Easements & Rights Laws,2020-05-28 15:08:38,2000-01-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<p>\r\n\tMaryland has a long-standing law prot...,NaN,0
4,42,0,1,2,44,534,OR02R,Solar and Wind Easements & Rights Laws & Local...,2020-05-29 17:46:32,2000-01-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<p>&#10;&#9;Oregon has several laws that prote...,NaN,0
6,2,0,1,2,44,534,AK01R,Solar Easements,2020-05-28 13:37:25,2000-01-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<p>\r\n\tAlaska&#39;s solar easement provision...,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22127,13,1,3,1,88,553,GA84F,TVA - Commercial Building Revitalization Program,2020-06-02 03:38:30,2020-06-02 00:00:00,...,TVA Energy Right,NaN,NaN,NaN,NaN,NaN,NaN,<p>TVA offers incentives for renovations of ex...,NaN,0
22128,38,1,3,1,88,553,NC100F,TVA - Commercial Building Revitalization Program,2020-06-02 03:41:00,2020-06-02 00:00:00,...,TVA Energy Right,NaN,NaN,NaN,NaN,NaN,NaN,<p>TVA offers incentives for renovations of ex...,NaN,0
22129,58,1,1,2,44,538,WY05R,Solar Rights,2020-06-04 11:45:59,2020-06-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"<p>Wyoming’s Solar Rights Act (1981), states t...",NaN,0


In [4]:
headers = []
for col in programdata.columns:
    headers.append(col)
    
headers

#or: headers = list(programdata.columns)

['state_id',
 'is_entire_state',
 'implementing_sector_id',
 'program_category_id',
 'program_type_id',
 'created_by_user_id',
 'code',
 'name',
 'updated_ts',
 'created_ts',
 'published',
 'websiteurl',
 'administrator',
 'fundingsource',
 'budget',
 'start_date',
 'start_date_text',
 'end_date',
 'end_date_text',
 'summary',
 'additional_technologies',
 'fromSir']

In [5]:
programdata_string = programdata.applymap(str)

In [6]:
programdata_string

,state_id,is_entire_state,implementing_sector_id,program_category_id,program_type_id,created_by_user_id,code,name,updated_ts,created_ts,...,administrator,fundingsource,budget,start_date,start_date_text,end_date,end_date_text,summary,additional_technologies,fromSir
id,,,,,,,,,,,,,,,,,,,,,
1,39,1,1,2,44,534,ND02R,Solar Easements,2020-06-01 17:35:33,2000-01-01 00:00:00,...,nan,nan,nan,nan,nan,nan,nan,<p>&#10;&#9;North Dakota's solar easement law ...,nan,0
2,4,1,1,1,32,534,AZ10F,Qualifying Wood Stove Deduction,2017-03-21 20:07:14,2000-01-01 00:00:00,...,nan,nan,nan,1994-01-01 05:00:00,1/1/1994,nan,nan,<div>&#10;&#9;This incentive allows Arizona ta...,nan,0
3,24,1,1,2,44,534,MD01R,Solar Easements & Rights Laws,2020-05-28 15:08:38,2000-01-01 00:00:00,...,nan,nan,nan,nan,nan,nan,nan,<p>\r\n\tMaryland has a long-standing law prot...,nan,0
4,42,0,1,2,44,534,OR02R,Solar and Wind Easements & Rights Laws & Local...,2020-05-29 17:46:32,2000-01-01 00:00:00,...,nan,nan,nan,nan,nan,nan,nan,<p>&#10;&#9;Oregon has several laws that prote...,nan,0
6,2,0,1,2,44,534,AK01R,Solar Easements,2020-05-28 13:37:25,2000-01-01 00:00:00,...,nan,nan,nan,nan,nan,nan,nan,<p>\r\n\tAlaska&#39;s solar easement provision...,nan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22127,13,1,3,1,88,553,GA84F,TVA - Commercial Building Revitalization Program,2020-06-02 03:38:30,2020-06-02 00:00:00,...,TVA Energy Right,nan,nan,nan,nan,nan,nan,<p>TVA offers incentives for renovations of ex...,nan,0
22128,38,1,3,1,88,553,NC100F,TVA - Commercial Building Revitalization Program,2020-06-02 03:41:00,2020-06-02 00:00:00,...,TVA Energy Right,nan,nan,nan,nan,nan,nan,<p>TVA offers incentives for renovations of ex...,nan,0
22129,58,1,1,2,44,538,WY05R,Solar Rights,2020-06-04 11:45:59,2020-06-04 00:00:00,...,nan,nan,nan,nan,nan,nan,nan,"<p>Wyoming’s Solar Rights Act (1981), states t...",nan,0


In [7]:
# contains_innovation = []
# for header in headers:
#     contains_innovation.append(programdata_string[programdata_string[header].str.contains('innovation')])
# contains_innovation

In [8]:
# contains_innovation_name = []
# for value in range(len(contains_innovation)):
#     contains_innovation_name.append(contains_innovation[value]['name'])
    
# contains_innovation_name


In [10]:
# contains_Innovation = []
# for header in headers:
#     contains_Innovation.append(programdata_string[programdata_string[header].str.contains('Innovation')])

# contains_Innovation_names = []
# for value in range(len(contains_Innovation)):
#     contains_Innovation_names.append(contains_Innovation[value]['name'])
    
# contains_Innovation_names


In [11]:
# contains_diversity = []
# for header in headers:
#     contains_diversity.append(programdata_string[programdata_string[header].str.contains('diversity')])

# contains_diversity_names = []
# for value in range(len(contains_diversity)):
#     contains_diversity_names.append(contains_diversity[value]['name'])

    
# contains_diversity_names

In [12]:
# contains_Diversity = []
# for header in headers:
#     contains_Diversity.append(programdata_string[programdata_string[header].str.contains('Diversity')])

# contains_Diversity_names = []
# for value in range(len(contains_Diversity)):
#     contains_Diversity_names.append(contains_Diversity[value]['name'])

    
# contains_Diversity_names

In [13]:
# contains_women = []
# for header in headers:
#     contains_women.append(programdata_string[programdata_string[header].str.contains('women')])

# contains_women_names = []
# for value in range(len(contains_women)):
#     contains_women_names.append(contains_women[value]['name'])

    
# contains_women_names

In [14]:
# contains_Women = []
# for header in headers:
#     contains_Women.append(programdata_string[programdata_string[header].str.contains('Women')])

# contains_Women_names = []
# for value in range(len(contains_Women)):
#     contains_Women_names.append(contains_Women[value]['name'])

    
# contains_Women_names

In [15]:
# This is the code to search for all of the key terms at once
#     through the entire dataframe

searchterms = ['Innovation', 'innovation', 'Diversity', 'diversity',
               'Diverse', 'diverse', 'Women', 'women', 'Woman',
              'woman', 'State Match', 'State match', 'state Match',
              'state match', 'Matching', 'matching']

contains_searchterm = []
for searchterm in searchterms:
    for header in headers:
        contains_searchterm.append(programdata_string[programdata_string[header].str.contains(searchterm)])
        
contains_searchterm_names = []

for value in range(len(contains_searchterm)):
    contains_searchterm_names.append(contains_searchterm[value]['name'])

    
contains_searchterm_names






[Series([], Name: name, dtype: object),
 Series([], Name: name, dtype: object),
 Series([], Name: name, dtype: object),
 Series([], Name: name, dtype: object),
 Series([], Name: name, dtype: object),
 Series([], Name: name, dtype: object),
 Series([], Name: name, dtype: object),
 id
 175          Energy Innovations Small Grant (EISG) Program
 1646                   Community Innovations Grant Program
 3266     Edison Innovation Clean Energy Manufacturing F...
 3678                   Edison Innovation Clean Energy Fund
 3758     Lincoln Electric System (LES) - Commercial and...
 4327     Tennessee Solar Institute - Solar Innovation G...
 4848             Edison Innovation Green Growth Fund Loans
 4856     Small Wind Innovation Zone Program and Model O...
 4931                          Innovation Incentive Program
 5122     NYSERDA - Innovation in Manufacturing Clean En...
 22074                      Energy Innovation Grant Program
 Name: name, dtype: object,
 Series([], Name: name, dtyp

In [16]:
# # All program IDs and Names that contain at least one 
# #      of the following search terms:

# # 'Innovation', 'innovation', 'Diversity', 'diversity',
# #                'Diverse', 'diverse', 'Women', 'women', 'Woman',
# #               'woman', 'State Match', 'State match', 'state Match',
# #               'state match', 'Matching', 'matching'




#  id
#  175          Energy Innovations Small Grant (EISG) Program
#  1646                   Community Innovations Grant Program
#  3266     Edison Innovation Clean Energy Manufacturing F...
#  3678                   Edison Innovation Clean Energy Fund
#  3758     Lincoln Electric System (LES) - Commercial and...
#  4327     Tennessee Solar Institute - Solar Innovation G...
#  4848             Edison Innovation Green Growth Fund Loans
#  4856     Small Wind Innovation Zone Program and Model O...
#  4931                          Innovation Incentive Program
#  5122     NYSERDA - Innovation in Manufacturing Clean En...
#  22074                      Energy Innovation Grant Program
#  Name: name, dtype: object,
 
#  id
#  1236                     Operational Demonstration Program
#  3597             Guidance for Local Wind Energy Ordinances
#  5122     NYSERDA - Innovation in Manufacturing Clean En...
#  22074                      Energy Innovation Grant Program
#  Name: name, dtype: object,


#  id
#  175          Energy Innovations Small Grant (EISG) Program
#  721                                   Fuel Cell Initiative
#  1236                     Operational Demonstration Program
#  1646                   Community Innovations Grant Program
#  1901     Riverside Public Utilities - Energy Efficiency...
#  2482     U.S. Virgin Islands - Discretionary Grant Program
#  3266     Edison Innovation Clean Energy Manufacturing F...
#  3678                   Edison Innovation Clean Energy Fund
#  3758     Lincoln Electric System (LES) - Commercial and...
#  4327     Tennessee Solar Institute - Solar Innovation G...
#  4848             Edison Innovation Green Growth Fund Loans
#  4856     Small Wind Innovation Zone Program and Model O...
#  4861     City of Indianapolis - Green Building Incentiv...
#  4879                               Grants to Green Program
#  4931                          Innovation Incentive Program
#  5860                               Clean Energy Fund (CEF)
#  5861                               Clean Energy Fund (CEF)
#  22074                      Energy Innovation Grant Program
#  Name: name, dtype: object,

#  id
#  175         Energy Innovations Small Grant (EISG) Program
#  336     CCEF - Renewable Energy Projects in Pre-Develo...
#  4856    Small Wind Innovation Zone Program and Model O...
#  Name: name, dtype: object,
 
#  id
#  175          Energy Innovations Small Grant (EISG) Program
#  355      Illinois Clean Energy Community Foundation Grants
#  453                                           Net Metering
#  552                      Self-Generation Incentive Program
#  1217                        Energy Star Home Grant Program
#  1606     Renewable Energy and Energy-Efficient Technolo...
#  1996            City of Ashland - Green Building Incentive
#  2444     Northern Plains EC - Commercial Energy Efficie...
#  2485                            Green Building Requirement
#  2863     City of Costa Mesa - Fee Waiver for Green Buil...
#  2877                    North Carolina Green Business Fund
#  3279     Qualifying Advanced Energy Manufacturing Inves...
#  4856     Small Wind Innovation Zone Program and Model O...
#  5558     City of Aspen and Pitkin County - Renewable En...
#  5860                               Clean Energy Fund (CEF)
#  5861                               Clean Energy Fund (CEF)
#  5883                                 Clean Energy Standard
#  21826    Seventhwave (formerly Energy Center of Wiscons...
#  21863                  Energy Efficiency Resource Standard
#  Name: name, dtype: object,


#  id
#  47      Emerging Technology Development Credit
#  613          Fuel Mix and Emissions Disclosure
#  1263              On-Site Renewable DG Program
#  3969              Enterprise Energy Fund Loans
#  4003              Sunburst Solar Grant Program
#  4551             Enterprise Energy Fund Grants
#  Name: name, dtype: object,


#  id
#  2266     Diverse Power - Energy Efficient New Construct...
#  4828     Diverse Power - Energy Efficient Existing Home...
#  21032                          Diverse Power - Green Power
#  Name: name, dtype: object,
 
#  id
#  2266     Diverse Power - Energy Efficient New Construct...
#  4828     Diverse Power - Energy Efficient Existing Home...
#  21032                          Diverse Power - Green Power
#  Name: name, dtype: object,


#  id
#  2266     Diverse Power - Energy Efficient New Construct...
#  4828     Diverse Power - Energy Efficient Existing Home...
#  21032                          Diverse Power - Green Power
#  Name: name, dtype: object,


#  id
#  2266     Diverse Power - Energy Efficient New Construct...
#  4828     Diverse Power - Energy Efficient Existing Home...
#  21032                          Diverse Power - Green Power
#  Name: name, dtype: object,


#  id
#  3656    Sustainable Energy Utility (SEU) - Energy Star...
#  3982    Sustainable Energy Utility (SEU) - Standard Li...
#  Name: name, dtype: object,
 


#  id
#  5107     City of Phoenix - Energize Phoenix Residential...
#  20802    Ohio Department of Development - Advanced Ener...
#  Name: name, dtype: object,
 

#  id
#  1039    City of San Francisco - Solar Program
#  Name: name, dtype: object,

#  id
#  4455    School Energy Efficiency Grant Program
#  Name: name, dtype: object,
 
#  id
#  332                 MTC - Matching Grants for Communities
#  4602    Oncor Electric Delivery - City and School Matc...
#  Name: name, dtype: object,
 
#  id
#  1096                   Value-Added Producer Grant Program
#  2757                       Renewable Energy Grant Program
#  4455               School Energy Efficiency Grant Program
#  4602    Oncor Electric Delivery - City and School Matc...
#  4894    Georgia Interfaith Power and Light - Energy Im...
#  Name: name, dtype: object,

#  id
#  332    MTC - Matching Grants for Communities
#  Name: name, dtype: object,
 
#  id
#  3005    Local Small Wind Rebate Programs
#  Name: name, dtype: object,
 
#  id
#  31                                  BEF - Solar 4R Schools
#  62                                  BEF - Solar 4R Schools
#  228               Alternative Power & Energy Grant Program
#  332                  MTC - Matching Grants for Communities
#  351      Pennsylvania Energy Development Authority (PED...
#  680      Sustainable Development Fund Grant Program (PE...
#  1111                                BEF - Solar 4R Schools
#  1185                Small Business Advantage Grant Program
#  1191                               Lighting Grants Program
#  1192               Energy Efficiency in Historic Buildings
#  1193          Solar and Wind Powered Stock Pump Initiative
#  1606     Renewable Energy and Energy-Efficient Technolo...
#  1901     Riverside Public Utilities - Energy Efficiency...
#  2029     College Station Utilities - Residential Energy...
#  2335        Small Business Non-Profit Energy Audit Program
#  2714                   Florida Farm to Fuel Grants Program
#  2926                              Local PV Rebate Programs
#  2927             Local Solar Water Heating Rebate Programs
#  2936                      Local Insulation Rebate Programs
#  2957         We Energies - Solar Thermal Incentive Program
#  2979     New Jersey Natural Gas (NJNG) - Home Performan...
#  3005                      Local Small Wind Rebate Programs
#  3080                        Renewable Energy Grant Program
#  3220                Wind and Geothermal Incentives Program
#  3266     Edison Innovation Clean Energy Manufacturing F...
#  3352                       Solar Energy Incentives Program
#  3354          High Performance Building Incentives Program
#  3585     Empire Electric Association - Residential Ener...
#  3599                Wind and Geothermal Incentives Program
#  3601                Wind and Geothermal Incentives Program
#  3602          High Performance Buildings Incentive Program
#  3603                       Solar Energy Incentives Program
#  3604                       Solar Energy Incentives Program
#  3636       DCEO - Community Renewable Energy Grant Program
#  3648                  Alternative and Clean Energy Program
#  3650                  Alternative and Clean Energy Program
#  3651                  Alternative and Clean Energy Program
#  3678                   Edison Innovation Clean Energy Fund
#  3700      Solar Energy Legacy Grants for Local Governments
#  3742          Renewable Energy Demonstration Pilot Program
#  3743     Department of Rural Affairs - Wind Energy for ...
#  3900     Clean Energy Development Fund (CEDF) - Municip...
#  4455                School Energy Efficiency Grant Program
#  4602     Oncor Electric Delivery - City and School Matc...
#  4626     Longmont Power & Communications - Residential ...
#  4879                               Grants to Green Program
#  4929     River Falls Municipal Utilities - Non-Profit E...
#  4931                          Innovation Incentive Program
#  4975                                      EnergyFit Nevada
#  5039               Commonwealth Organics-to-Energy Program
#  5163     Empire Electric Association - Commercial Energ...
#  5289                City of Lowell - BLEU Projects Program
#  5340                          BetterBuildings for Michigan
#  5586     Michigan Accelerating Technologies (MATch) Ene...
#  5754       Wind and Geothermal Industry Incentives Program
#  5763     Xcel Energy - Small Business Energy Efficiency...
#  5884     City of Louisville - Green Infrastructure Ince...
#  5915     City of Louisville - Green Infrastructure Ince...
#  5945                             Solar Energy Loan Program
#  21963    Oklahoma Municipal Power Authority - Demand an...
#  Name: name, dtype: object,
 